In [73]:
import sys

In [ ]:
print(sys.path)

['/Users/nmolivo/Documents/Repos/case_scrape/notebooks', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/nmolivo/Documents/Repos/case_scrape/.venv/lib/python3.10/site-packages']


: 

In [ ]:
sys.path.append("/Users/nmolivo/Documents/Repos/case_scrape")

: 

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker, scoped_session
from chromedriver_py import binary_path
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy.util.compat import contextmanager
from selenium.common.exceptions import NoSuchElementException
import random
from typing import Tuple


: 

In [202]:
from case_scrape.case_scrape import models

In [195]:
from case_scrape.case_scrape.models import Base
from case_scrape.case_scrape.db.db_config import DBConfig


def init_db():

    engine = DBConfig(
        password=os.getenv("DB_PASSWORD"),
        user=os.getenv("DB_USER")
    ).get_engine()

    Base.metadata.create_all(bind=engine)

    print("Initialized the db")

def delete_db():

    engine = DBConfig(
        password=os.getenv("DB_PASSWORD"),
        user=os.getenv("DB_USER")
    ).get_engine()

    Base.metadata.drop_all(bind=engine)

    print("Cleared the db")

In [197]:
init_db()

Initialized the db


In [198]:
engine = DBConfig(
        password=os.getenv("DB_PASSWORD"),
        user=os.getenv("DB_USER")
    ).get_engine()

Session = sessionmaker(bind=engine)
session = Session()



In [199]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
opts = Options()
# opts.headless = True
driver = webdriver.Chrome(options=opts, executable_path=binary_path)

/var/folders/rb/4k3rm5dn3_5gbmbfgfhgdy280000gn/T/ipykernel_37947/1583674229.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=opts, executable_path=binary_path)


In [194]:
driver.close()

In [ ]:
range_start = 624667

: 

In [200]:
case_numbers = [str(x) for x in range(655656, 666532)] + \
[str(x) for x in range(647313, 655655)] + \
[str(x) for x in range(635863, 647312)] + \
[str(x) for x in range(624667, 635860)] + \
[str(x) for x in range(612912, 624666)] + \
[str(x) for x in range(602355, 612910)] 

In [201]:
for case_no_str in case_numbers:
    driver = search_case_number(driver, case_no_str)
    data_elems = driver.find_elements(By.XPATH, "//*[(@id='SheetContentPlaceHolder_ctl00_gvCaseResults')]//*[@href]")
    if len(data_elems) > 0:
        for i, elem in enumerate(data_elems):
            data_elems = driver.find_elements(By.XPATH, "//*[(@id='SheetContentPlaceHolder_ctl00_gvCaseResults')]//*[@href]")
            data_elems[i].click()
            driver, case_number = fetch_case_summary(session, driver)
            driver, case_number = fetch_case_summary_tables(session, driver, case_number)
            driver, case_number = fetch_docket_info(session, driver, case_number)
            driver.back()
            driver = check_tos(driver)
            driver, case_number = fetch_cost_info(session, driver, case_number)
            driver.back()
            driver = check_tos(driver)
            driver, case_number = fetch_defendant_info(session, driver, case_number)
            driver.back()
            driver = check_tos(driver)
            driver, case_number = fetch_attorney_info(session, driver, case_number)
            driver.back()
            driver = check_tos(driver)
            driver.back()
    else:
        driver, case_number = fetch_case_summary(session, driver)
        driver, case_number = fetch_case_summary_tables(session, driver, case_number)
        driver, case_number = fetch_docket_info(session, driver, case_number)
        driver, case_number = fetch_cost_info(session, driver, case_number)
        driver, case_number = fetch_defendant_info(session, driver, case_number)
        driver, case_number = fetch_attorney_info(session, driver, case_number)
    

    

NoSuchTableError: alias

In [185]:
from re import I
import sqlalchemy


# opts = Options()
# # opts.headless = True
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)#, executable_path=binary_path)

def check_tos(driver):
    # anticipates terms of service pop up
    if len(driver.find_elements(By.ID, "SheetContentPlaceHolder_btnYes")) > 0:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "SheetContentPlaceHolder_btnYes"))).click()
        driver.implicitly_wait(3)
    return driver

def search_case_number(driver: WebDriver, case_number: str) -> webdriver.Chrome:
    """Goes to search page, upon landing on the search page, this function will navigate the agreement and then enter a case 
    number to search criminal records and click the submit button.
    Be sure to already be on the case search page when you run this function."""
    driver.get("https://cpdocket.cp.cuyahogacounty.us/Search.aspx") 
    driver = check_tos(driver)
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_rbCrCase')]"))).click()
    driver.implicitly_wait(2)
    
    if len(driver.find_elements(By.ID, "SheetContentPlaceHolder_btnYes")) > 0:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "SheetContentPlaceHolder_btnYes"))).click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_rbCrCase')]"))).click()
        driver.implicitly_wait(3)
    
    # checks if searching elements are there
    if len(driver.find_elements(By.ID, "SheetContentPlaceHolder_ctl00_gvCaseResults")) == 0:
        
        text_box = driver.find_element(By.XPATH, "//input[(@type = 'text')]")
        text_box.clear()
        text_box.send_keys(case_number)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "SheetContentPlaceHolder_criminalCaseSearch_btnSubmitCase"))).click()
        driver.implicitly_wait(random.randint(4,15))

    return driver

def get_table(case_number: str, header_xpath: str, data_xpath: str) -> pd.DataFrame:
    headers = driver.find_elements(By.XPATH, header_xpath)
    data = driver.find_elements(By.XPATH, data_xpath)
    data_list = [d.text for d in data]
    table = {}
    for i, h in enumerate(headers):
        table[h.text.upper()]=data_list[i::len(headers)]
    df = pd.DataFrame(table)
    df["CASE_NUMBER"] = case_number
    return df

def write_pandas_df_to_db(db, df: pd.DataFrame, table_to_write_to: str):
    engine = db.get_bind()
    listToWrite = df.to_dict(orient='records')
    conn = engine.connect()

    metadata = sqlalchemy.schema.MetaData(bind=engine)
    table = sqlalchemy.Table(table_to_write_to, metadata, autoload=True)

    # Open the session
    Session = sessionmaker(bind=engine)
    session = Session()

    # Inser the dataframe into the database in one bulk
    conn.execute(table.insert(), listToWrite)

def fetch_case_summary(db, driver) -> str:
    """commits returns case_number"""
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblCaseNumber')]")))
    case_number = driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblCaseNumber')]").text
    case_information=models.CaseInformation(
        case_number= case_number,
        status=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblCaseStatus')]").text,
        judge_name=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblJudgeName')]").text,
        next_event=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblNextEvent')]").text,
        arrested_date=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblArrested')]").text,
        arresting_agency=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblArrestingAgency')]").text,
        arresting_agency_report=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblArestingAgencyReport')]").text,
        court_of_appeals_case=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblCOACase')]").text
    )
    db.add(case_information)
    defendant_info = models.DefendantInformation(
        case_number=case_number,
        number=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblNumber')]").text,
        name=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblName')]").text,
        status=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_defStatus_lblDefStatus')]").text,
        date_of_birth=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblDOB')]").text,
        race=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblRace')]").text,
        sex=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_lblSex')]").text,
        other_cases=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_otherCases_lblOtherCases')]").text,
        co_defendants=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_caseSummary_otherDefs_lblCoDefs')]").text
    ) 
    db.add(defendant_info)  
    db.commit()
    return driver, case_number

def fetch_case_summary_tables(db, driver, case_number) -> Tuple[WebDriver, str]:
    """gets info from case summary page"""
    #additional tables
    charge_df = get_table(case_number, 
                        header_xpath="//table[(@id = 'SheetContentPlaceHolder_caseCharges_gvCharges')]//th", 
                        data_xpath="//table[(@id = 'SheetContentPlaceHolder_caseCharges_gvCharges')]//td")
    write_pandas_df_to_db(db, charge_df, "charges")
    
    bond_df = get_table(case_number, 
                        header_xpath="//table[(@id = 'SheetContentPlaceHolder_caseBondInfo_gvBonds')]//th", 
                        data_xpath="//table[(@id = 'SheetContentPlaceHolder_caseBondInfo_gvBonds')]//td")
    write_pandas_df_to_db(db, bond_df, "bond_information")
    
    action_df = get_table(case_number,
                        header_xpath="//table[(@id = 'SheetContentPlaceHolder_caseActions_gvActions')]//th",
                        data_xpath="//table[(@id = 'SheetContentPlaceHolder_caseActions_gvActions')]//td")
    write_pandas_df_to_db(db, action_df, "case_actions")
    return driver, case_number


def fetch_docket_info(db, driver, case_number):
    """from case summary page, clicks docket, downloads information"""
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "SheetContentPlaceHolder_caseHeader_lbDocket"))).click()
    driver = check_tos(driver)
    docket_table = get_table(case_number,
    header_xpath="//table[(@id = 'SheetContentPlaceHolder_caseDocket_gvDocket')]//th",
    data_xpath="//table[(@id = 'SheetContentPlaceHolder_caseDocket_gvDocket')]//td")
    write_pandas_df_to_db(db, docket_table, "docket_information")
    return driver, case_number

def fetch_cost_info(db, driver, case_number):
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.ID,"SheetContentPlaceHolder_caseHeader_lbCosts"))).click()
    # anticipates terms of service
    driver = check_tos(driver)
    #SheetContentPlaceHolder_caseCosts_gvCosts
    cost_table = get_table(case_number, header_xpath="//table[(@id = 'SheetContentPlaceHolder_caseCosts_gvCosts')]//th", data_xpath="//table[(@id = 'SheetContentPlaceHolder_caseCosts_gvCosts')]//td")
    write_pandas_df_to_db(db, cost_table, "cost_information")
    return driver, case_number
        
def fetch_defendant_info(db, driver, case_number):
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.ID,"SheetContentPlaceHolder_caseHeader_lbDefendant"))).click()
    driver = check_tos(driver)
    def_id=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defGeneral_lblDefID')]").text
    defendent_info = models.Defendant(
        #SheetContentPlaceHolder_defGeneral_defStatus_lblDefStatus
        name=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defGeneral_lblName')]").text,
        id=def_id,
        case_number = case_number,
        status=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defGeneral_defStatus_lblDefStatus')]").text,
        marital_status=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defGeneral_lblMaritalStatus')]").text,
        birth_city=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defGeneral_lblBirthCity')]").text,
        citizenship=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defGeneral_lblCitizenship')]").text,
        address_1=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defAddress_lblAddress1')]").text,
        address_2=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defAddress_lblAddress2')]").text,
        address_3=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defAddress_lblCSZ')]").text,
        race=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblRace')]").text,
        height=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblHeight')]").text,
        sex=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblSex')]").text,
        weight=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblWeight')]").text,
        age=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblAge')]").text,
        eyes=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblEyes')]").text,
        date_of_birth=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblDOB')]").text,
        hair=driver.find_element(By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_defIDChar_lblHair')]").text,
        )
    # anticipates terms of service
    db.add(defendent_info)
    db.commit()
    alias_table = get_table(case_number, header_xpath="//table[(@id = 'SheetContentPlaceHolder_defAlias_gvAlias')]//th", data_xpath="//table[(@id = 'SheetContentPlaceHolder_defAlias_gvAlias')]//td")
    alias_table["defendent_id"] = def_id
    write_pandas_df_to_db(db, alias_table, "alias")
    return driver, case_number


def get_attorney_table(db, driver, case_number, table_data):
    data = driver.find_elements(By.XPATH, table_data)
    data_list = [d.text for d in data]
    atts = [x for x in list(set(data_list)) if x.count("Address/Phone") ==1 and x.count("Attorney Name") ==1]
    attorney_names = [x.split("\nAddress/Phone:")[0].replace("Attorney Name:", "").strip() for x in atts]
    attorney_address = [x.split("\nAddress/Phone:")[1].split("\nPh:")[0].strip() for x in atts]
    attorney_phone = [x.split("\nAddress/Phone:")[1].split("\nPh:")[1].strip() for x in atts]
    for n, a, p in zip(attorney_names, attorney_address, attorney_phone):
        attorney = models.Attorney(
            case_number=case_number,
            name=n,
            address=a,
            phone=p
        )
        db.add(attorney)
    db.commit()
    return driver

def fetch_attorney_info(db, driver, case_number):
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.ID,"SheetContentPlaceHolder_caseHeader_lbAttorney"))).click()
    driver = check_tos(driver)
    driver = get_attorney_table(db, driver, case_number, "//table[(@id='SheetContentPlaceHolder_attyInfo_gvAttyInfo')]//*")
    return driver, case_number




        
    


# h-list

In [ ]:
already_scraped = list(df["CASE_NUMBER"].unique())

: 

In [ ]:
new_cases = ["618926",
"620395",
"616872",
"620865",
"614781",
"628724",
"631170",
"627746",
"635840",
"633911",
"646343",
"646590",
"641595",
"641983",
"647080",
"650202",
"650473",
"649886",
"648192",
"648166",
"666430",
"657457",
"663574",
"660005",
"665568",
"616263",
"613712",
"618748",
"615216",
"619363",
"625226",
"625071",
"633620",
"627879",
"627879",
"644646",
"645807",
"640586",
"646354",
"642321",
"652069",
"651583",
"655333",
"653793",
"653048",
"660255",
"662286",
"662347",
"656397",
"660002",
"619447",
"621879",
"619539",
"619626",
"619180",
"628630",
"627817",
"633075",
"629819",
"627173",
"640255",
"645111",
"646461",
"636695",
"644912",
"655597",
"653681",
"651825",
"654631",
"651363",
"656323",
"665547",
"659751",
"662658",
"662176",
"619386",
"619384",
"619846",
"623628",
"619478",
"626510",
"634952",
"631300",
"635457",
"630824",
"643838",
"645684",
"636488",
"642792",
"636996",
"652397",
"648833",
"648433",
"653455",
"647816",
"664281",
"664518",
"664801",
"662304",
"664021"]

: 